In [1]:
with open('baseline.test.pred.txt') as f:
    lines = f.readlines()

In [15]:
baseline_dev_texts = []
with open('baseline.dev.pred.txt') as f:
    lines = f.readlines()
    for line in tqdm(lines):
        baseline_dev_texts += [line]

  0%|          | 0/7374 [00:00<?, ?it/s]

In [1]:
import json

In [2]:

from pathlib import Path
from datasets import Dataset, load_dataset, Split
from data.dataset.data_augmentations import flatten_conversation

data_dir = Path("resources/bart/")

data_files = {
    Split.TRAIN: str((data_dir / "train.history_belief").absolute()),
    Split.VALIDATION: str((data_dir / "val.history_belief").absolute()),
    Split.TEST: str((data_dir / "test.history_belief").absolute()),
}

dataset = load_dataset(
    "data/dataset/multiwoz_dataset.py", data_files=data_files
)
dataset = dataset.map(
    flatten_conversation,
    batched=True,
    remove_columns=dataset["train"].column_names,
)

Using custom data configuration default-fa91ae44cff1e0ff
Reusing dataset multi_woz_dataset (/data/users/cting3/.cache/huggingface/datasets/multi_woz_dataset/default-fa91ae44cff1e0ff/0.0.0/f68a68638b040ca98e35780d893ba6dd439629d12a3b62bdda1ba2e3d7a88647)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /data/users/cting3/.cache/huggingface/datasets/multi_woz_dataset/default-fa91ae44cff1e0ff/0.0.0/f68a68638b040ca98e35780d893ba6dd439629d12a3b62bdda1ba2e3d7a88647/cache-a1c988fe37507e5f.arrow
Loading cached processed dataset at /data/users/cting3/.cache/huggingface/datasets/multi_woz_dataset/default-fa91ae44cff1e0ff/0.0.0/f68a68638b040ca98e35780d893ba6dd439629d12a3b62bdda1ba2e3d7a88647/cache-339ba30f507aa4aa.arrow
Loading cached processed dataset at /data/users/cting3/.cache/huggingface/datasets/multi_woz_dataset/default-fa91ae44cff1e0ff/0.0.0/f68a68638b040ca98e35780d893ba6dd439629d12a3b62bdda1ba2e3d7a88647/cache-aeb133aae4fea810.arrow


In [3]:
from tqdm.auto import tqdm
from postprocessing import postprocessing

In [7]:

split="test"
results = {}
turn_id = 0
for prediction_text in tqdm(lines):
    gold_text = dataset[split]["turn"][turn_id]

    dialogue_id = dataset[split]["conversation_id"][turn_id]
    if dialogue_id not in results.keys():
        results[dialogue_id] = {
            "generated_turn_belief": [],
            "target_turn_belief": [],
        }

    results[dialogue_id]["generated_turn_belief"] += [
        postprocessing(prediction_text)
    ]
    results[dialogue_id]["target_turn_belief"] += [
        postprocessing(gold_text)
    ]

    turn_id += 1


  0%|          | 0/7372 [00:00<?, ?it/s]

In [17]:
split="validation"
baseline_dev_results = {}
turn_id = 0
for prediction_text in tqdm(baseline_dev_texts):
    gold_text = dataset[split]["turn"][turn_id]

    dialogue_id = dataset[split]["conversation_id"][turn_id]
    if dialogue_id not in baseline_dev_results.keys():
        baseline_dev_results[dialogue_id] = {
            "generated_turn_belief": [],
            "target_turn_belief": [],
        }

    baseline_dev_results[dialogue_id]["generated_turn_belief"] += [
        postprocessing(prediction_text)
    ]
    baseline_dev_results[dialogue_id]["target_turn_belief"] += [
        postprocessing(gold_text)
    ]

    turn_id += 1


  0%|          | 0/7374 [00:00<?, ?it/s]

In [18]:
with open("baseline_dev_pred.json", "w") as outfile:
    json.dump(baseline_dev_results, outfile)

In [25]:
adapter_no_c_dev_texts = []
with open('adapter_no_c.dev.pred.txt') as f:
    lines = f.readlines()
    for line in tqdm(lines):
        adapter_no_c_dev_texts += [line]
        
split="validation"
adapter_no_c_dev_results = {}
turn_id = 0
for prediction_text in tqdm(adapter_no_c_dev_texts):
    gold_text = dataset[split]["turn"][turn_id]

    dialogue_id = dataset[split]["conversation_id"][turn_id]
    if dialogue_id not in adapter_no_c_dev_results.keys():
        adapter_no_c_dev_results[dialogue_id] = {
            "generated_turn_belief": [],
            "target_turn_belief": [],
        }

    adapter_no_c_dev_results[dialogue_id]["generated_turn_belief"] += [
        postprocessing(prediction_text)
    ]
    adapter_no_c_dev_results[dialogue_id]["target_turn_belief"] += [
        postprocessing(gold_text)
    ]

    turn_id += 1
    
with open("adapter_no_c_dev_pred.json", "w") as outfile:
    json.dump(adapter_no_c_dev_results, outfile)

  0%|          | 0/7374 [00:00<?, ?it/s]

  0%|          | 0/7374 [00:00<?, ?it/s]

In [26]:
adapter_no_c_test_texts = []
with open('adapter_no_c.test.pred.txt') as f:
    lines = f.readlines()
    for line in tqdm(lines):
        adapter_no_c_test_texts += [line]
        
split="test"
adapter_no_c_test_results = {}
turn_id = 0
for prediction_text in tqdm(adapter_no_c_test_texts):
    gold_text = dataset[split]["turn"][turn_id]

    dialogue_id = dataset[split]["conversation_id"][turn_id]
    if dialogue_id not in adapter_no_c_test_results.keys():
        adapter_no_c_test_results[dialogue_id] = {
            "generated_turn_belief": [],
            "target_turn_belief": [],
        }

    adapter_no_c_test_results[dialogue_id]["generated_turn_belief"] += [
        postprocessing(prediction_text)
    ]
    adapter_no_c_test_results[dialogue_id]["target_turn_belief"] += [
        postprocessing(gold_text)
    ]

    turn_id += 1
    
with open("adapter_no_c_test_pred.json", "w") as outfile:
    json.dump(adapter_no_c_test_results, outfile)

  0%|          | 0/7372 [00:00<?, ?it/s]

  0%|          | 0/7372 [00:00<?, ?it/s]

In [2]:
len(lines)

7372

In [9]:
import json

In [5]:
# with open("baseline_test_pred.json", "w") as outfile:
#     json.dump(results, outfile)

In [27]:

with open('baseline_dev_pred.json') as f:
    dev_baseline_results = json.load(f)
with open('baseline_test_pred.json') as f:
    test_baseline_results = json.load(f)
    
with open('adapter_no_c_dev_pred.json') as f:
    dev_adapter_no_c_results = json.load(f)
with open('adapter_no_c_test_pred.json') as f:
    test_adapter_no_c_results = json.load(f)
    

In [28]:
from tqdm.auto import tqdm

from utils.Constants import SLOT_VALS
from utils.dst import ignore_none, default_cleaning

slot_template = {slot: "" for slot in SLOT_VALS}


def get_slot_map(slot_triplet_str_list):
    slot_map = slot_template.copy()
    for slot_triplet_str in slot_triplet_str_list:
        slot_triplets = slot_triplet_str.split()
        key = slot_triplets[0] + " " + slot_triplets[1]
        val = slot_triplets[2:]
        if key not in SLOT_VALS:
            continue
        slot_map[key] = val
    return slot_map

def get_unique_slot_map(preds, targets):
    unique_slots = set()
    pred_map = {}
    target_map = {}
    
    for pred_str in preds:
        triplet = pred_str.split()
        key = triplet[0] + " " + triplet[1]
        val = triplet[2:]
        if key not in SLOT_VALS: continue
        pred_map[key] = val
        unique_slots.add(key)
    
    for target_str in targets:
        triplet = target_str.split()
        key = triplet[0] + " " + triplet[1]
        val = triplet[2:]
        if key not in SLOT_VALS: continue
        target_map[key] = val
        unique_slots.add(key)
    
    return unique_slots.copy(), pred_map.copy(), target_map.copy()


def evaluate_dst(input_results):
    num_turns = 0
    joint_acc = 0
    slot_acc = 0
    r_slot_acc = 0

    num_slots = len(SLOT_VALS)
    num_r_slots = 0

    clean_tokens = ["<s>", "</s>"]
    
    results = input_results.copy()
    
    for dial in tqdm(results.keys()):
        dialogue_pred = results[dial]["generated_turn_belief"]
        dialogue_target = results[dial]["target_turn_belief"]

        for turn_id, (turn_target, turn_pred) in enumerate(
            zip(dialogue_target, dialogue_pred)
        ):
            
            # clean
            for bs in turn_pred:
                if bs in clean_tokens + ["", " "] or bs.split()[-1] == "none":
                    turn_pred.remove(bs)

            new_turn_pred = []
            for bs in turn_pred:
                for tok in clean_tokens:
                    bs = bs.replace(tok, "").strip()
                    new_turn_pred.append(bs)
            turn_pred = new_turn_pred

            turn_pred, turn_target = ignore_none(turn_pred, turn_target)
            # print(turn_pred, turn_target)
            turn_pred, turn_target = default_cleaning(turn_pred, turn_target)

            join_flag = False

            # calculate joint accuracy
            if set(turn_target) == set(turn_pred):
                joint_acc += 1
                join_flag = True

            pred_slot_map = get_slot_map(turn_pred)
            target_slot_map = get_slot_map(turn_target)

            # calculate slot accuracy
            for slot_key in SLOT_VALS:
                if target_slot_map[slot_key] == pred_slot_map[slot_key]:
                    slot_acc += 1

            # calculate relative slot accuracy
            (
                unique_slots,
                unique_pred_map,
                unique_target_map,
            ) = get_unique_slot_map(turn_pred, turn_target)
            for slot_key in unique_slots:
                if slot_key not in unique_target_map.keys():
                    continue
                if slot_key not in unique_pred_map.keys():
                    continue
                if unique_target_map[slot_key] == unique_pred_map[slot_key]:
                    r_slot_acc += 1
            num_r_slots += len(unique_slots)

            num_turns += 1
    joint_acc /= num_turns
    slot_acc /= num_slots * num_turns
    r_slot_acc /= num_r_slots
    print("joint accuracy: {}".format(joint_acc))
    print("slot accuracy: {}".format(slot_acc))
    print("relative slot accuracy: {}".format(r_slot_acc))
    return {
        "joint_acc": joint_acc,
        "slot_acc": slot_acc,
        "r_slot_acc": r_slot_acc,
    }



In [29]:
score = evaluate_dst(dev_baseline_results)
with open("baseline_dev_score.json", "w") as outfile:
    json.dump(score, outfile)

score = evaluate_dst(test_baseline_results)
with open("baseline_test_score.json", "w") as outfile:
    json.dump(score, outfile)
    
score = evaluate_dst(dev_adapter_no_c_results)
with open("adapter_no_c_dev_score.json", "w") as outfile:
    json.dump(score, outfile)

score = evaluate_dst(test_adapter_no_c_results)
with open("adapter_no_c_test_score.json", "w") as outfile:
    json.dump(score, outfile)

  0%|          | 0/1000 [00:00<?, ?it/s]

joint accuracy: 0.012747491185245458
slot accuracy: 0.8551580531422522
relative slot accuracy: 0.0


  0%|          | 0/999 [00:00<?, ?it/s]

joint accuracy: 0.014378730330982095
slot accuracy: 0.8514606269581503
relative slot accuracy: 0.0


  0%|          | 0/1000 [00:00<?, ?it/s]

joint accuracy: 0.392731217792243
slot accuracy: 0.9668451490415322
relative slot accuracy: 0.7899157334516022


  0%|          | 0/999 [00:00<?, ?it/s]

joint accuracy: 0.3724905046120456
slot accuracy: 0.9640225438888208
relative slot accuracy: 0.7746471152528437


In [ ]:

score = evaluate_dst(dev_adapter_no_c_results)

In [11]:
len("phil".split())

1

In [24]:
score = evaluate_dst(test_adapter_no_c_results)
with open("adapter_no_c_test_pred.json", "w") as outfile:
    json.dump(score, outfile)

  0%|          | 0/999 [00:00<?, ?it/s]

joint accuracy: 0.3724905046120456
slot accuracy: 0.9640225438888208
relative slot accuracy: 0.7746471152528437
